# 필요 데이터 생성

In [60]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
import warnings
warnings.filterwarnings('ignore')

In [61]:
### 사고유형위험도데이터 생성


df = pd.read_csv('Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='CP949',engine='python')

# temp) 총부상자수 변수
df['총부상자수'] = df['사상자수'] - df['사망자수']


total= df.groupby(['발생지시도','발생지시군구']).count().주야.sort_index()
a = df.groupby(['발생지시도','발생지시군구']).sum().사망자수.sort_index()
b = df.groupby(['발생지시도','발생지시군구']).sum().총부상자수.sort_index()
temp=(a*12+b*3)/total

for i,j in temp.index:
    df.loc[(df.발생지시도==i)&(df.발생지시군구==j),'발생지시군구_위험도'] = temp[i,j]

#df.발생지시군구_위험도.describe()

# 발생지시도x요일x주야_위험도

total = df.groupby(['발생지시도','요일','주야']).count().도로형태.sort_index()
a = df.groupby(['발생지시도','요일','주야']).sum().사망자수.sort_index()
b = df.groupby(['발생지시도','요일','주야']).sum().총부상자수.sort_index()
temp = (a*12+b*3)/total

for i,j,k in temp.index:
    df.loc[(df.발생지시도==i)&(df.요일==j)&(df.주야==k),'발생지시도x요일x주야_위험도'] = temp[i,j,k]

#df.발생지시도x요일x주야_위험도.describe()

# 차량조합_위험도

total = df.groupby(['당사자종별_1당_대분류','당사자종별_2당_대분류']).count().사고유형_중분류
a = df.groupby(['당사자종별_1당_대분류','당사자종별_2당_대분류']).sum().사망자수.sort_index()
b = df.groupby(['당사자종별_1당_대분류','당사자종별_2당_대분류']).sum().총부상자수.sort_index()
temp = (a*12+b*3)/total

for i,j in temp.index:
    df.loc[(df.당사자종별_1당_대분류==i)&(df.당사자종별_2당_대분류==j),'차량조합_위험도'] = temp[i,j]

#df.차량조합_위험도.describe()

# 사고유형_위험도

total = df.groupby(['사고유형_중분류']).count().당사자종별_1당_대분류
a = df.groupby(['사고유형_중분류']).sum().사망자수.sort_index()
b = df.groupby(['사고유형_중분류']).sum().총부상자수.sort_index()
temp = (a*12+b*3)/total

for i in temp.index:
    df.loc[(df.사고유형_중분류==i),'사고유형_위험도'] = temp[i]


temp = df[['발생지시도','발생지시군구','요일','주야','당사자종별_1당_대분류','당사자종별_2당_대분류',
   '사고유형_중분류','발생지시군구_위험도','발생지시도x요일x주야_위험도','차량조합_위험도',
   '사고유형_위험도']]

temp.to_csv('4_danger.csv',encoding='CP949',index=False)

In [62]:
### NMF 데이터 생성

df = pd.read_csv('Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='CP949',engine='python')

# 발생지변수 생성

df['발생지'] = df['발생지시도'] + df['발생지시군구']
df['발생지'].replace('경남창원시(통합)','경남창원시', inplace=True)

df['요일/사고유형_중'] = df['요일'] + '_'+df['사고유형_중분류']

day_acc = df.groupby(['발생지','요일/사고유형_중'])['주야'].count().unstack().fillna(0)

X = day_acc.values

model = NMF(n_components=3, init='random', random_state=0)
W = model.fit_transform(X)
df_nmf = pd.DataFrame(W)
df_nmf.rename(columns={0:'day_acc_nmf1',1:'day_acc_nmf2',2:'day_acc_nmf3'}, inplace=True)
df_nmf = df_nmf.join(pd.DataFrame(day_acc.index))

df = df.merge(df_nmf, on='발생지')
df.drop('요일/사고유형_중',axis=1,inplace=True)

temp = df[['day_acc_nmf1','day_acc_nmf2','day_acc_nmf3','발생지']]

temp.columns = ['요일_사고유형_NMF1','요일_사고유형_NMF2','요일_사고유형_NMF3','발생지']
temp = temp.groupby('발생지').mean()
temp.to_csv('5_nmf.csv',encoding='CP949',index=True)

In [63]:
## 사고다발지 데이터 생성
df = pd.read_csv('Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='CP949',engine='python')

df_jaywalk = pd.read_csv('무단횡단사고다발지(2012~2016).csv',encoding='CP949',engine='python')
df_elder = pd.read_csv('보행노인사고다발지(2012~2016).csv',encoding='CP949',engine='python')
df_child = pd.read_csv('보행어린이사고다발지(2012~2016).csv',encoding='CP949',engine='python')
df_school = pd.read_csv('스쿨존내어린이사고다발지(2012~2016).csv',encoding='CP949',engine='python')
df_bike = pd.read_csv('자전거사고다발지(2012~2016).csv',encoding='CP949',engine='python')

df_jaywalk["사고피해정도"] = df_jaywalk.사망자수*4 + df_jaywalk.중상자수*3 + df_jaywalk.경상자수*2 + df_jaywalk.부상신고자수*1
df_elder["사고피해정도"] = df_elder.사망자수*4 + df_elder.중상자수*3 + df_elder.경상자수*2 + df_elder.부상신고자수*1
df_child["사고피해정도"] = df_child.사망자수*4 + df_child.중상자수*3 + df_child.경상자수*2 + df_child.부상신고자수*1
df_bike["사고피해정도"] = df_bike.사망자수*4 + df_bike.중상자수*3 + df_bike.경상자수*2 + df_bike.부상신고자수*1
df_school["사고피해정도"] = df_school.사망자수*4 + df_school.중상자수*3 + df_school.경상자수*2 + df_school.부상신고자수*1

def replace_region(df):    
    df = df.groupby('다발지명').agg({'사고피해정도': 'sum', '발생건수':'sum'})
    df.reset_index(inplace=True)
    df['시도'] = df['다발지명'].apply(lambda x: x.split(' ')[0])
    df['시군구'] = df['다발지명'].apply(lambda x: x.split(' ')[1])
    df['시도'].replace(['경상남도', '경상북도', '전라남도', '전라북도', '충청남도', '충청북도'],
                                   ['경남', '경북', '전남', '전북', '충남', '충북'], inplace=True)
    df['시도'] = df['시도'].apply(lambda x: x[:2])
    df.drop('다발지명', axis=1, inplace=True)
    df['시군구'].replace('세종특별자치시','세종', inplace=True)
    df['발생지'] = df['시도'] + df['시군구']
    df = df.groupby(['발생지'])['사고피해정도', '발생건수'].sum()
    df.reset_index(inplace=True)
    return df
    
walk_list = ['2_mudan','2_old','2_child','2_school','2_bic']
i = 0
for x in [df_jaywalk, df_elder, df_child, df_school, df_bike]:
    df = replace_region(x)
    df.to_csv(walk_list[i]+'.csv',index=False)
    i += 1

# 피쳐추가

In [64]:
## 기존 데이터 불러오기 ###
df = pd.read_csv('Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='CP949',engine='python')
df = df[['주야', '요일', '사망자수', '사상자수', '중상자수', '경상자수', '부상신고자수', '발생지시도', '발생지시군구',
       '사고유형_대분류', '사고유형_중분류', '법규위반', '도로형태_대분류', '도로형태', '당사자종별_1당_대분류',
       '당사자종별_2당_대분류']]

### 추가 데이터 불러오기 ###
# 고속도로 데이터 #
highway = pd.read_csv('외부데이터_고속도로정보.csv',encoding='CP949',engine='python')
highway['총길이']=np.float64(highway.총길이)

# 사고다발지 데이터 #
jaywalk = pd.read_csv('무단횡단사고다발지(2012~2016).csv',encoding='CP949',engine='python')
elder = pd.read_csv('보행노인사고다발지(2012~2016).csv',encoding='CP949',engine='python')
child = pd.read_csv('보행어린이사고다발지(2012~2016).csv',encoding='CP949',engine='python')
bike = pd.read_csv('자전거사고다발지(2012~2016).csv',encoding='CP949',engine='python')
school = pd.read_csv('스쿨존내어린이사고다발지(2012~2016).csv',encoding='CP949',engine='python')

jaywalk.columns = ['발생지', '무단횡단피해', '무단횡단건수']
elder.columns = ['발생지', '노인피해', '노인건수']
child.columns = ['발생지', '어린이피해', '어린이건수']
bike.columns = ['발생지', '자전거피해', '자전거건수']
school.columns = ['발생지', '학교피해', '학교건수']

# 위험도 데이터 #
danger = pd.read_csv('2_danger.csv',encoding='CP949')

# nmf 데이터 #
nmf = pd.read_csv('2_nmf.csv',encoding='CP949')

### 기존데이터 전처리 ###

df.loc[df.발생지시군구=='연기군','발생지시군구']='세종'
df.loc[(df.발생지시도=='충남')&(df.발생지시군구=='세종'),'발생지시도']='세종'

# 충북 청원군 -> 충북 청주시 변경 (14년 7월)
df.loc[df.발생지시군구=='청원군','발생지시군구']='청주시'

#발생지 변수
df['발생지']=df['발생지시도']+df['발생지시군구']
df['발생지'].replace('경남창원시(통합)','경남창원시',inplace=True)



ValueError: Length mismatch: Expected axis has 15 elements, new values have 3 elements

In [ ]:
def change(df, highway, jaywalk, elder, child, bike, school, danger, nmf ):
    ### 고속도로데이터 ###
    df=pd.merge(df,highway,on='발생지',how='left')

    ### 사고다발지데이터 ###
    for x in  [jaywalk, elder, child, bike, school]:
        df = df.merge(x, on='발생지', how='left')
    df.drop(df[(df.주야.isnull())&(df.요일.isnull())].index,inplace=True)

    df.loc[(df.발생지.isnull()==False) & (df.무단횡단피해.isnull()==True), ['무단횡단피해', '무단횡단건수']]=0
    df.loc[(df.발생지.isnull()==False) & (df.노인피해.isnull()==True), ['노인피해', '노인건수']]=0
    df.loc[(df.발생지.isnull()==False) & (df.어린이피해.isnull()==True), ['어린이피해','어린이건수']]=0
    df.loc[(df.발생지.isnull()==False) & (df.자전거피해.isnull()==True), ['자전거피해','자전거건수']]=0
    df.loc[(df.발생지.isnull()==False) & (df.학교피해.isnull()==True), ['학교피해','학교건수']]=0
    
    ### nmf 데이터 ###
    df = df.merge(nmf, on='발생지',how='left')

    ### 위험도 데이터 ###
    # 발생지시군구 위험도
    temp = danger.groupby(['발생지시도','발생지시군구']).mean().발생지시군구_위험도
    for i,j in temp.index:
        df.loc[(df.발생지시도==i)&(df.발생지시군구==j),'발생지시군구_위험도'] = temp[i,j]

    # 발생지시도x요일x주야 위험도
    temp = danger.groupby(['발생지시도','요일','주야']).mean().발생지시도x요일x주야_위험도
    for i,j,k in temp.index:
        df.loc[(df.발생지시도==i)&(df.요일==j)&(df.주야==k),'발생지시도x요일x주야_위험도'] = temp[i,j,k]

    # 차량조합 위험도
    temp = danger.groupby(['당사자종별_1당_대분류','당사자종별_2당_대분류']).mean().차량조합_위험도
    for i,j in temp.index:
        df.loc[(df.당사자종별_1당_대분류==i)&(df.당사자종별_2당_대분류==j),'차량조합_위험도'] = temp[i,j]

    # 사고유형 위험도
    temp = danger.groupby(['사고유형_중분류']).mean().사고유형_위험도
    for i in temp.index:
        df.loc[(df.사고유형_중분류==i),'사고유형_위험도'] = temp[i]

    # 사고위험 평균
    df['위험도평균'] = np.nanmean(np.array(df[['발생지시군구_위험도','발생지시도x요일x주야_위험도','차량조합_위험도','사고유형_위험도']]),axis=1)

    
    return df

In [ ]:
def binning1(df):
    # 피해자수
    ### - 사망자수
    df.loc[(df.사망자수>=3)&(df.사망자수<=4),'사망자수'] = 3
    df.loc[(df.사망자수>=5),'사망자수'] = 6
    
    ### - 중상자수
    df.loc[(df.중상자수>=3)&(df.중상자수<=4),'중상자수'] = 3
    df.loc[(df.중상자수>=5)&(df.중상자수<=9),'중상자수'] = 6
    df.loc[(df.중상자수>=10),'중상자수'] = 13

    ### - 경상자수

    df.loc[(df.경상자수>=1)&(df.경상자수<=2),'경상자수'] = 1
    df.loc[(df.경상자수>=3),'경상자수'] = 5

    ### - 부상신고자수

    df.loc[(df.부상신고자수>=2)&(df.부상신고자수<=3),'부상신고자수'] = 2
    df.loc[(df.부상신고자수>=4)&(df.부상신고자수<=8),'부상신고자수'] = 5
    df.loc[(df.부상신고자수>=9),'부상신고자수'] = 13

    ### - 사상자수

    df['사상자수'] = df['사망자수'] + df['중상자수'] + df['경상자수'] +df['부상신고자수']
    
    df.loc[(df.사상자수>=4)&(df.사상자수<=8),'사상자수'] = 5
    df.loc[(df.사상자수>=9),'사상자수'] = 13

    # 고속도로
    ### - 총길이
    a = df[df.총길이!=0].총길이.describe()[4]
    b = df[df.총길이!=0].총길이.describe()[5]
    c = df[df.총길이!=0].총길이.describe()[6]

    df.loc[(df.총길이>0)&(df.총길이<=a),'총길이'] = 1
    df.loc[(df.총길이>a)&(df.총길이<=b),'총길이'] = 2
    df.loc[(df.총길이>b)&(df.총길이<=c),'총길이'] = 3
    df.loc[(df.총길이>c),'총길이'] = 4


    ### - 제한속도
    df.loc[(df['제한속도']>0)&(df['제한속도']<=100),'제한속도'] = 1
    df.loc[(df['제한속도']>100),'제한속도'] = 2


    # 사고다발지
    ### - 무단횡단피해
    df.loc[(df.무단횡단피해>0)&(df.무단횡단피해<=100),'무단횡단피해'] = 1
    df.loc[(df.무단횡단피해>100)&(df.무단횡단피해<=250),'무단횡단피해'] = 2
    df.loc[(df.무단횡단피해>250)&(df.무단횡단피해<=475),'무단횡단피해'] = 3
    df.loc[(df.무단횡단피해>475)&(df.무단횡단피해<=650),'무단횡단피해'] = 4
    df.loc[(df.무단횡단피해>650),'무단횡단피해'] = 5

    ### - 무단횡단건수
    df.loc[(df.무단횡단건수>0)&(df.무단횡단건수<=25),'무단횡단건수'] = 1
    df.loc[(df.무단횡단건수>25)&(df.무단횡단건수<=50),'무단횡단건수'] = 2
    df.loc[(df.무단횡단건수>50)&(df.무단횡단건수<=125),'무단횡단건수'] = 3
    df.loc[(df.무단횡단건수>125)&(df.무단횡단건수<=200),'무단횡단건수'] = 4
    df.loc[(df.무단횡단건수>200),'무단횡단건수'] = 5

    ### - 노인피해
    df.loc[(df.노인피해>0)&(df.노인피해<=70),'노인피해'] = 1
    df.loc[(df.노인피해>70)&(df.노인피해<=200),'노인피해'] = 2
    df.loc[(df.노인피해>200)&(df.노인피해<=400),'노인피해'] = 3
    df.loc[(df.노인피해>400)&(df.노인피해<=600),'노인피해'] = 4
    df.loc[(df.노인피해>600),'노인피해'] = 5

    ### - 노인건수
    df.loc[(df.노인건수>0)&(df.노인건수<=25),'노인건수'] = 1
    df.loc[(df.노인건수>25)&(df.노인건수<=75),'노인건수'] = 2
    df.loc[(df.노인건수>75)&(df.노인건수<=150),'노인건수'] = 3
    df.loc[(df.노인건수>150)&(df.노인건수<=250),'노인건수'] = 4
    df.loc[(df.노인건수>250),'노인건수'] = 5

    ### - 어린이피해
    df.loc[(df.어린이피해>0)&(df.어린이피해<=20),'어린이피해'] = 1
    df.loc[(df.어린이피해>20)&(df.어린이피해<=60),'어린이피해'] = 2
    df.loc[(df.어린이피해>60)&(df.어린이피해<=100),'어린이피해'] = 3
    df.loc[(df.어린이피해>100)&(df.어린이피해<=180),'어린이피해'] = 4
    df.loc[(df.어린이피해>180),'어린이피해'] = 5

    ### - 어린이건수
    df.loc[(df.어린이건수>0)&(df.어린이건수<=10),'어린이건수'] = 1
    df.loc[(df.어린이건수>10)&(df.어린이건수<=25),'어린이건수'] = 2
    df.loc[(df.어린이건수>25)&(df.어린이건수<=50),'어린이건수'] = 3
    df.loc[(df.어린이건수>50)&(df.어린이건수<=80),'어린이건수'] = 4
    df.loc[(df.어린이건수>80),'어린이건수'] = 5

    ### - 자전거피해
    df.loc[(df.자전거피해>0)&(df.자전거피해<=70),'자전거피해'] = 1
    df.loc[(df.자전거피해>70)&(df.자전거피해<=270),'자전거피해'] = 2
    df.loc[(df.자전거피해>270)&(df.자전거피해<=650),'자전거피해'] = 3
    df.loc[(df.자전거피해>650)&(df.자전거피해<=1000),'자전거피해'] = 4
    df.loc[(df.자전거피해>1000),'자전거피해'] = 5

    ### - 자전거건수
    df.loc[(df.자전거건수>0)&(df.자전거건수<=30),'자전거건수'] = 1
    df.loc[(df.자전거건수>30)&(df.자전거건수<=100),'자전거건수'] = 2
    df.loc[(df.자전거건수>100)&(df.자전거건수<=300),'자전거건수'] = 3
    df.loc[(df.자전거건수>300)&(df.자전거건수<=500),'자전거건수'] = 4
    df.loc[(df.자전거건수>500),'자전거건수'] = 5

    ### - 학교피해
    df.loc[(df.학교피해>0)&(df.학교피해<=7),'학교피해'] = 1
    df.loc[(df.학교피해>7)&(df.학교피해<=15),'학교피해'] = 2
    df.loc[(df.학교피해>15)&(df.학교피해<=30),'학교피해'] = 3
    df.loc[(df.학교피해>30)&(df.학교피해<=50),'학교피해'] = 4
    df.loc[(df.학교피해>50),'학교피해'] = 5

    ### - 학교피해건수
    df.loc[(df.학교건수>0)&(df.학교건수<=2),'학교건수'] = 1
    df.loc[(df.학교건수>2)&(df.학교건수<=5),'학교건수'] = 2
    df.loc[(df.학교건수>5)&(df.학교건수<=9),'학교건수'] = 3
    df.loc[(df.학교건수>9)&(df.학교건수<=17),'학교건수'] = 4
    df.loc[(df.학교건수>17),'학교건수'] = 5
    
    
    # 위험도변수
    ### - 발생지시군구_위험도
    df.loc[(df.발생지시군구_위험도>0)&(df.발생지시군구_위험도<=14),'발생지시군구_위험도'] = 1
    df.loc[(df.발생지시군구_위험도>14)&(df.발생지시군구_위험도<=15.5),'발생지시군구_위험도'] = 2
    df.loc[(df.발생지시군구_위험도>15.5)&(df.발생지시군구_위험도<=17),'발생지시군구_위험도'] = 3
    df.loc[(df.발생지시군구_위험도>17)&(df.발생지시군구_위험도<=19),'발생지시군구_위험도'] = 4
    df.loc[(df.발생지시군구_위험도>19),'발생지시군구_위험도'] = 5

    ### - 발생지시도x요일x주야_위험도
    df.loc[(df.발생지시도x요일x주야_위험도>0)&(df.발생지시도x요일x주야_위험도<=14),'발생지시도x요일x주야_위험도'] = 1
    df.loc[(df.발생지시도x요일x주야_위험도>14)&(df.발생지시도x요일x주야_위험도<=15),'발생지시도x요일x주야_위험도'] = 2
    df.loc[(df.발생지시도x요일x주야_위험도>15)&(df.발생지시도x요일x주야_위험도<=16.5),'발생지시도x요일x주야_위험도'] = 3
    df.loc[(df.발생지시도x요일x주야_위험도>16.5)&(df.발생지시도x요일x주야_위험도<=19),'발생지시도x요일x주야_위험도'] = 4
    df.loc[(df.발생지시도x요일x주야_위험도>19),'발생지시도x요일x주야_위험도'] = 5

    ### - 차량조합_위험도
    df.loc[(df.차량조합_위험도>0)&(df.차량조합_위험도<=12.5),'차량조합_위험도'] = 1
    df.loc[(df.차량조합_위험도>12.5)&(df.차량조합_위험도<=14),'차량조합_위험도'] = 2
    df.loc[(df.차량조합_위험도>14)&(df.차량조합_위험도<=17),'차량조합_위험도'] = 3
    df.loc[(df.차량조합_위험도>17)&(df.차량조합_위험도<=21),'차량조합_위험도'] = 4
    df.loc[(df.차량조합_위험도>21),'차량조합_위험도'] = 5

    ### - 사고유형_위험도
    df.loc[(df.사고유형_위험도>0)&(df.사고유형_위험도<=13.5),'사고유형_위험도'] = 1
    df.loc[(df.사고유형_위험도>13.5)&(df.사고유형_위험도<=15),'사고유형_위험도'] = 2
    df.loc[(df.사고유형_위험도>15)&(df.사고유형_위험도<=16),'사고유형_위험도'] = 3
    df.loc[(df.사고유형_위험도>16)&(df.사고유형_위험도<=17.5),'사고유형_위험도'] = 4
    df.loc[(df.사고유형_위험도>17.5),'사고유형_위험도'] = 5

    ### - 위험도평균
    df.loc[(df.위험도평균>0)&(df.위험도평균<=13.5),'위험도평균'] = 1
    df.loc[(df.위험도평균>13.5)&(df.위험도평균<=14.5),'위험도평균'] = 2
    df.loc[(df.위험도평균>14.5)&(df.위험도평균<=16.5),'위험도평균'] = 3
    df.loc[(df.위험도평균>16.5)&(df.위험도평균<=18),'위험도평균'] = 4
    df.loc[(df.위험도평균>18),'위험도평균'] = 5
    
    ### -nmf 변수
    mask0 = (df.요일_사고유형_NMF1==0)
    mask1 = (df.요일_사고유형_NMF1>0)&(df.요일_사고유형_NMF1<=0.5)
    mask2 = (df.요일_사고유형_NMF1>0.5)&(df.요일_사고유형_NMF1<=1)
    mask3 = (df.요일_사고유형_NMF1>1)&(df.요일_사고유형_NMF1<=1.5)
    mask4 = (df.요일_사고유형_NMF1>1.5)&(df.요일_사고유형_NMF1<=3)
    mask5 = (df.요일_사고유형_NMF1>3)
    df.loc[mask0,'temp'] = 0
    df.loc[mask1,'temp'] = 1
    df.loc[mask2,'temp'] = 2
    df.loc[mask3,'temp'] = 3
    df.loc[mask4,'temp'] = 4
    df.loc[mask5,'temp'] = 5
    df['요일_사고유형_NMF1'] = df['temp']
    df.drop('temp',axis=1,inplace=True)
    
    mask0 = (df.요일_사고유형_NMF2==0)
    mask1 = (df.요일_사고유형_NMF2>0)&(df.요일_사고유형_NMF2<=0.7)
    mask2 = (df.요일_사고유형_NMF2>0.7)&(df.요일_사고유형_NMF2<=1.3)
    mask3 = (df.요일_사고유형_NMF2>1.3)&(df.요일_사고유형_NMF2<=1.7)
    mask4 = (df.요일_사고유형_NMF2>1.7)&(df.요일_사고유형_NMF2<=2.6)
    mask5 = (df.요일_사고유형_NMF2>2.6)
    df.loc[mask0,'temp'] = 0
    df.loc[mask1,'temp'] = 1
    df.loc[mask2,'temp'] = 2
    df.loc[mask3,'temp'] = 3
    df.loc[mask4,'temp'] = 4
    df.loc[mask5,'temp'] = 5
    df['요일_사고유형_NMF2'] = df['temp']
    df.drop('temp',axis=1,inplace=True)

    
    mask0 = (df.요일_사고유형_NMF3==0)
    mask1 = (df.요일_사고유형_NMF3>0)&(df.요일_사고유형_NMF3<=0.3)
    mask2 = (df.요일_사고유형_NMF3>0.3)&(df.요일_사고유형_NMF3<=0.7)
    mask3 = (df.요일_사고유형_NMF3>0.7)&(df.요일_사고유형_NMF3<=1.3)
    mask4 = (df.요일_사고유형_NMF3>1.3)&(df.요일_사고유형_NMF3<=2)
    mask5 = (df.요일_사고유형_NMF3>2)
    df.loc[mask0,'temp'] = 0
    df.loc[mask1,'temp'] = 1
    df.loc[mask2,'temp'] = 2
    df.loc[mask3,'temp'] = 3
    df.loc[mask4,'temp'] = 4
    df.loc[mask5,'temp'] = 5
    df['요일_사고유형_NMF3'] = df['temp']
    df.drop('temp',axis=1,inplace=True)

    # object 변환
    df_num = df.select_dtypes(['float', 'int'])
    for col in df_num.columns:
        df[col] = df_num[col].astype(object)
    
    return df

In [ ]:
df = change(df, highway, jaywalk, elder, child, bike, school, danger, nmf )

In [ ]:
temp.to_csv('original_kor.csv',encoding='CP949',index=False)